In [4]:
!pip install numpy
!pip install h5py

   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   --------------------------- ------------ 2.1/3.0 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 3.0/3.0 MB 9.7 MB/s eta 0:00:00


In [5]:
import h5py
import numpy as np


def load_data():
    train_dataset = h5py.File('dataset/trainset.hdf5', "r")
    X_train = np.array(train_dataset["X_train"][:]) # your train set features
    y_train = np.array(train_dataset["Y_train"][:]) # your train set labels

    test_dataset = h5py.File('dataset/testset.hdf5', "r")
    X_test = np.array(test_dataset["X_test"][:]) # your train set features
    y_test = np.array(test_dataset["Y_test"][:]) # your train set labels
    
    return X_train, y_train, X_test, y_test

x_train, y_train, x_test, y_test = load_data()

In [9]:
# Normalize image vectors
x_train = x_train / 255.
x_test = x_test / 255.

# Normalize labels pas necessaire car les labels sont déjà normalisés
# flatten les image
x_train = x_train.reshape(x_train.shape[0], 4096) # 64x64 = 4096 ou -1 pour laisser numpy calculer
x_test = x_test.reshape(x_test.shape[0], 4096)

# afficher les dimensions
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (1000, 4096)
y_train shape: (1000, 1)
x_test shape: (200, 4096)
y_test shape: (200, 1)


In [11]:
# generer random weights et bias
w = np.random.randn(4096, 1)
b = np.random.randn(1)

# afficher les dimensions
print("w shape:", w.shape)
print("b shape:", b.shape)

# initialiser w et b
def initialize_with_zeros(nb_weights=4096):
    w = np.random.randn(nb_weights, 1)
    b = np.random.randn(1)
    return w, b

w shape: (4096, 1)
b shape: (1,)


In [12]:
!pip install torch

  Using cached networkx-3.2.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached MarkupSafe-2.1.5-cp39-cp39-win_amd64.whl.metadata (3.1 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ---------------------------------------- 0.0/199.3 MB ? eta -:--:--
   ---------------------------------------- 0.5/199.3 MB 4.2 MB/s eta 0:00:48
   ---------------------------------------- 1.8/199.3 MB 4.8 MB/s eta 0:00:42
    --------------------------------------- 3.1/199.3 MB 4.9 MB/s eta 0:00:41
    --------------------------------------- 4.2/199.3 MB 4.8 MB/s eta 0:00:41
   - -------------------------------------- 5.2/199.3 MB 4.8 MB/s eta 0:00:41
   - -------------------------------------- 6.6/199.3 MB 4.9 MB/s eta 0:00:40
   - -------------------------------------- 7.6/199.3 MB 4.8 MB/s eta 0:00:40
   - -------------------------------------- 8.9/199.3 MB 4.9 MB/s eta 0:00:40
   - -----------------------------------

In [14]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.8 MB 3.7 MB/s eta 0:00:02
   -------- ------------------------------- 1.6/7.8 MB 3.6 MB/s eta 0:00:02
   ------------- -------------------------- 2.6/7.8 MB 4.0 MB/s eta 0:00:02
   ------------------ --------------------- 3.7/7.8 MB 4.2 MB/s eta 0:00:01
   ------------------------ --------------- 4.7/7.8 MB 4.3 MB/s eta 0:00:01
   ------------------------------ --------- 6.0/7.8 MB 4.5 MB/s eta 0:00:01
   ------------------------------------- -- 7.3/7.8 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 4.5 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ------------------ --------------------- 1.0/2.2 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 4.8 MB

In [13]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

def model(X, w, b):
    """
    fonction qui prédit la classe d'une image
    @param X: image flatten
    @param w: poids
    @param b: bias
    @return: probabilité que l'image appartienne à la classe 1
    """
    z = np.dot(X, w) + b # produit matriciel et b use broadcasting pour combler b
    a = 1 / (1 + np.exp(-z)) # sigmoid car je veux une probabilité entre 0 et 1 (deux classes : si < 0.5 alors 0 sinon 1)
    return a

def log_loss(a, y):
    """
    fonction qui calcule la log loss (bceloss)
    @param a: probabilité que l'image appartienne à la classe 1
    @param y: vraie classe de l'image
    @return: log loss
    """
    return  nn.BCELoss(a,y)

def gradient(X, A, y):
    """Fonction qui calcule le gradient de la fonction de cout par rapport à w et b en utilsant le principe de la chaine afin de decomposer le calcule de depart qui est derive du cout par rapport a w ou b car effectivmenet, on a pas directement w1 et b dans la fonctio nde cout donc on doit decomposer le calcule(voir explication plus bas).
    @param X: image flatten
    @param a: probabilité que l'image appartienne à la classe 1
    @param y: vraie classe de l'image
    @return: matrice des gradient de la fonction de cout par rapport à w et idem pour b
    ca nous donne les gradient pour chaque poids et bias qu servi a mettre a jour les poids et bias plus tard
    """
    # derive de la fonction de cout par rapport à w et b calculé au prealable et ca donne ca
    # attention, si la fonction de perte/cout change ou que la fonction d activation change, il faut recalculer le gradient ou use pytorch, autograd ou keras qui le fait otut seul
    dW = 1 / len(y) * np.dot(X.T, A - y)
    db = 1 / len(y) * np.sum(A - y)
    return dW, db

def update(dW, db, W, b, learning_rate):
    """
    fonction qui met à jour les poids et le bias
    @param dW: gradient de la fonction de cout par rapport à w
    @param db: gradient de la fonction de cout par rapport à b
    @param W: poids
    @param b: bias
    @param learning_rate: taux d'apprentissage (pas donc la vitesse a laquelle on apprend)
    @return: nouveaux poids et bias
    """
    W = W - learning_rate * dW
    b = b - learning_rate * db
    return (W, b)

def train(X, y, learning_rate, epochs=1000):
    """
    fonction qui entraine le modèle
    @param X: images flatten
    @param y: vraies classes des images
    @param w: poids
    @param b: bias
    @param learning_rate: taux d'apprentissage
    @param epochs: nombre d'itérations
    @return: poids et bias entrainés
    """
    # init model
    loss = []
    w, b = initialize_with_zeros()
    # boucle d entrainement
    for epoch in range(epochs):
        # prédire la classe
        A = model(X, w, b)
        # calculer la fonction de cout
        loss = log_loss(A, y)
        # calculer le gradient
        dW, db = gradient(X, A, y)
        # mettre à jour les poids
        w, b = update(dW, db, w, b, learning_rate)
        # afficher la loss et epoch
        print(f"Epoch {epoch} - loss: {loss}")
        # save loss
        loss.append((loss, epoch))
    return w, b, loss

# --------------------------------entrainer le modèle----------------------------
w, b, loss = train(x_train, y_train, 0.01, 1000)

# afficher evolution loss
# Créer la figure et tracer la courbe
plt.figure(figsize=(8, 5))  # Taille de la figure
plt.plot(np.array(loss)[:, :1], np.array(loss)[:, 1:2], label='y = x^2', linestyle='-', marker='o')  # Tracer y en fonction de x

# Ajouter un titre et des labels
plt.title("Courbe de y = x^2")
plt.xlabel("Valeur de la perte")
plt.ylabel("Valeur de l'epoch")

# Ajouter une légende
plt.legend()

# Afficher la grille pour faciliter la lecture du graphique
plt.grid(True)

# Afficher le graphique
plt.show()

# --------------------------------tester le modèle----------------------------
def predcit(X, w, b, y):
    """
    fonction qui prédit la classe d'une image
    @param X: image flatten
    @param w: poids
    @param b: bias
    @return: classe prédite
    """
    A = model(X, w, b)
    # calculer la fonction de cout
    loss = log_loss(A, y)
    return loss
    
def predict_one_value(x, w, b):
    """
    fonction qui prédit la classe d'une image
    @param x: image flatten
    @param w: poids
    @param b: bias
    @return: classe prédite
    """
    # prédire la classe
    A = model(x, w, b)
    # si la probabilité est supérieure à 0.5, on prédit 1 sinon 0
    return A > 0.5


    




In [16]:
list_ = [(1, 2), (3, 4), (5, 6)]
array_ = np.array(list_)
list_1 = array_[:, :1]
print(list_1)

[[1]
 [3]
 [5]]
